# access -- ITS_LIVE data access from object storage

> These are tools for accessing ITS_LIVE velocity data from object storage (s3 buckets) and preliminary organization of dataset 

In [ ]:
#| default_exp access

In [ ]:
#| hide
from nbdev.showdoc import *
import nbdev 
from nbdev import nbdev_export

In [ ]:
#| export 

import geopandas as gpd
from shapely.geometry import Point
import os
import numpy as np
import xarray as xr
import rioxarray as rxr
from shapely.geometry import Polygon
from shapely.geometry import Point

import json
import s3fs



In [ ]:
#| hide

itslive_catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')


In [ ]:
#| export
def read_in_s3(http_url, chunks='auto'):
    ''' does some string formatting from zarr url and returns xarray dataset
    '''

    s3_url = http_url.replace('http','s3')
    s3_url = s3_url.replace('.s3.amazonaws.com','')

    datacube = xr.open_dataset(s3_url, engine='zarr',
                               storage_options={'anon':True},
                               chunks=chunks)
    return datacube
    
def find_granule_by_zone(zone):
    '''returns every url from itslive catalog that corresponds to a granule falling in specified UTM zone
    '''

    catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')

    zone_cat = catalog.loc[catalog['epsg'] == zone]

    return zone_cat

def find_granule_by_point(input_point, label='single_point'):
    '''returns url for the granule (zarr datacube) containing a specified point. point must be passed in epsg:4326
    '''
    catalog = gpd.read_file('https://its-live-data.s3.amazonaws.com/datacubes/catalog_v02.json')

    #make shapely point of input point
    p = gpd.GeoSeries([Point(input_point[0], input_point[1])],crs='EPSG:4326')
    #make gdf of point
    gdf = gdf = gpd.GeoDataFrame({'label': f'{label}', 
                                  'geometry':p})

    #find row of granule 
    granule = catalog.sjoin(gdf, how='inner')

    url = granule['zarr_url'].values[0]

    return url  



In [ ]:
#| hide
test_zone = find_granule_by_zone(32645)
test_zone.head()

,fill-opacity,fill,roi_percent_coverage,geometry_epsg,datacube_exist,zarr_url,epsg,geometry
527,0.203590,red,79.640980,"{'type': 'Polygon', 'coordinates': [[[200000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,32645,"POLYGON ((83.94987 27.99140, 84.96593 28.01020..."
528,0.332840,red,66.716002,"{'type': 'Polygon', 'coordinates': [[[200000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,32645,"POLYGON ((83.92388 28.89278, 84.94858 28.91229..."
529,0.041077,red,95.892282,"{'type': 'Polygon', 'coordinates': [[[200000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,32645,"POLYGON ((83.89667 29.79401, 84.93041 29.81425..."
530,0.791605,red,20.839500,"{'type': 'Polygon', 'coordinates': [[[200000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,32645,"POLYGON ((83.86820 30.69510, 84.91140 30.71608..."
531,0.957564,red,4.243618,"{'type': 'Polygon', 'coordinates': [[[200000, ...",1,http://its-live-data.s3.amazonaws.com/datacube...,32645,"POLYGON ((83.83841 31.59604, 84.89151 31.61778..."


In [ ]:
#| hide
point = [95.180191, 30.645973]

In [ ]:
#| hide
test = find_granule_by_point(point, 'test_label')
test

'http://its-live-data.s3.amazonaws.com/datacubes/v02/N30E090/ITS_LIVE_vel_EPSG32646_G0120_X750000_Y3350000.zarr'

In [ ]:
#| hide
da = read_in_s3(test)

In [ ]:
#| hide
da

<xarray.Dataset>
Dimensions:                    (mid_date: 6554, y: 833, x: 833)
Coordinates:
  * mid_date                   (mid_date) datetime64[ns] 2021-05-13T04:30:01....
  * x                          (x) float64 7.001e+05 7.003e+05 ... 8e+05
  * y                          (y) float64 3.4e+06 3.4e+06 ... 3.3e+06 3.3e+06
Data variables: (12/54)
    acquisition_date_img1      (mid_date) datetime64[ns] dask.array<chunksize=(6554,), meta=np.ndarray>
    acquisition_date_img2      (mid_date) datetime64[ns] dask.array<chunksize=(6554,), meta=np.ndarray>
    autoRIFT_software_version  (mid_date) <U5 dask.array<chunksize=(6554,), meta=np.ndarray>
    chip_size_height           (mid_date, y, x) float32 dask.array<chunksize=(6554, 70, 70), meta=np.ndarray>
    chip_size_width            (mid_date, y, x) float32 dask.array<chunksize=(6554, 70, 70), meta=np.ndarray>
    date_center                (mid_date) datetime64[ns] dask.array<chunksize=(6554,), meta=np.ndarray>
    ...                         ...
    vy_error_mask              (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_error_modeled           (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_error_slow              (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_stable_shift            (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_stable_shift_mask       (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
    vy_stable_shift_slow       (mid_date) float64 dask.array<chunksize=(6554,), meta=np.ndarray>
Attributes: (12/18)
    GDAL_AREA_OR_POINT:         Area
    author:                     ITS_LIVE, a NASA MEaSUREs project (its-live.j...
    autoRIFT_parameter_file:    http://its-live-data.s3.amazonaws.com/autorif...
    datacube_software_version:  1.0
    date_created:               09-Jun-2022 09:28:39
    date_updated:               09-Jun-2022 09:28:39
    ...                         ...
    s3:                         s3://its-live-data/datacubes/v02/N30E090/ITS_...
    skipped_granules:           s3://its-live-data/datacubes/v02/N30E090/ITS_...
    time_standard_img1:         UTC
    time_standard_img2:         UTC
    title:                      ITS_LIVE datacube of image_pair velocities
    url:                        https://its-live-data.s3.amazonaws.com/datacu...